In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from census import Census
from us import states

In [11]:
raw_data_dir = "data/raw"

raw_acs_population_path = os.path.join(raw_data_dir, "ACSDT5Y2023.B01003_2025-10-09T231035", "ACSDT5Y2023.B01003-Data.csv")
raw_incidents_path = os.path.join(raw_data_dir, "Crime_Data_from_2020_to_Present.csv")

## Crime Incidents Data

In [12]:
# Read raw data
incidents_raw = pd.read_csv(raw_incidents_path)

In [4]:
# Columns in raw data
incidents_raw.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')

In [42]:
# Filter out invalid lat/lon values -- online it says locations errors or missings are encoded as 0, 0 for lat/lon
incidents = incidents_raw[~((incidents_raw['LAT'] == 0) & (incidents_raw['LON'] == 0))]

In [43]:
# Convert to GeoDataFrame
incidents_geo = gpd.GeoDataFrame(
    incidents,
    geometry=gpd.points_from_xy(incidents["LON"], incidents["LAT"]),
    crs="EPSG:4326"
)

## ZCTA Boundary Data

In [44]:
# Load ZCTA data from Census
# -- Using 2023 because this is the most recent census data year and they should match
zctas = gpd.read_file(
    "https://www2.census.gov/geo/tiger/TIGER2023/ZCTA520/tl_2023_us_zcta520.zip"
)
zctas = zctas.to_crs("EPSG:4326")
zctas = zctas.rename(columns={"ZCTA5CE20": "ZCTA"})
print(f"Loaded {len(zctas)} ZIP code areas in the US")

Loaded 33791 ZIP code areas in the US


## Join ZCTA Boundary Data to Crime Incidents based on Latitude/Longitude

In [45]:
# Spatial join to merge ZCTA code if the lat/lon point is within the ZCTA boundary
# -- Not exactly sure how polygons/shape files work tbh, but have done a similar process to load in ACS data at work
incidents_geo = incidents_geo.to_crs(zctas.crs)
incidents_zcta = gpd.sjoin(incidents_geo, zctas, how = "left", predicate = "within")

In [46]:
# Drop crimes that we can't map to a zip code
incidents_zcta = incidents_zcta[~incidents_zcta["ZCTA"].isnull()]

## Crime Incidents Data Prep

In [48]:
# Create violent/non-violent classification
# -- Taken from Vikram's code

# Make a list of crimes that we identified as Violent from one of the previous steps

violent_crimes = ['CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT', 'INTIMATE PARTNER - SIMPLE ASSAULT', 'ROBBERY', 'CRIMINAL HOMICIDE', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT', 'RAPE, FORCIBLE', 'MANSLAUGHTER, NEGLIGENT', 'BATTERY - SIMPLE ASSAULT', 'LYNCHING']

# Create a new Crime Type column with Violent and Non-Violent labels
incidents_zcta['Crime Type'] = np.where(incidents_zcta['Weapon Used Cd'].notnull() | incidents_zcta['Crm Cd Desc'].isin(violent_crimes), 'Violent', 'Non-Violent')

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,GEOID20,GEOIDFQ20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,Crime Type
0,211507896,04/11/2021 12:00:00 AM,11/07/2020 12:00:00 AM,845,15,N Hollywood,1502,2,354,THEFT OF IDENTITY,...,91605,860Z200US91605,B5,G6350,S,13908248.0,39061.0,+34.2073412,-118.4009730,Non-Violent
1,201516622,10/21/2020 12:00:00 AM,10/18/2020 12:00:00 AM,1845,15,N Hollywood,1521,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,91605,860Z200US91605,B5,G6350,S,13908248.0,39061.0,+34.2073412,-118.4009730,Violent
2,240913563,12/10/2024 12:00:00 AM,10/30/2020 12:00:00 AM,1240,9,Van Nuys,933,2,354,THEFT OF IDENTITY,...,91411,860Z200US91411,B5,G6350,S,5054787.0,0.0,+34.1785171,-118.4596299,Non-Violent
3,210704711,12/24/2020 12:00:00 AM,12/24/2020 12:00:00 AM,1310,7,Wilshire,782,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND ...,...,90034,860Z200US90034,B5,G6350,S,8031974.0,23525.0,+34.0305777,-118.3996127,Non-Violent
4,201418201,10/03/2020 12:00:00 AM,09/29/2020 12:00:00 AM,1830,14,Pacific,1454,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,90292,860Z200US90292,B5,G6350,S,5428684.0,2857097.0,+33.9760775,-118.4520658,Non-Violent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004986,252104112,02/02/2025 12:00:00 AM,02/02/2025 12:00:00 AM,130,21,Topanga,2103,2,946,OTHER MISCELLANEOUS CRIME,...,91304,860Z200US91304,B5,G6350,S,25183699.0,56316.0,+34.2277906,-118.6446066,Non-Violent
1004987,250404100,02/18/2025 12:00:00 AM,02/18/2025 12:00:00 AM,1000,4,Hollenbeck,479,2,237,CHILD NEGLECT (SEE 300 W.I.C.),...,90023,860Z200US90023,B5,G6350,S,11181971.0,142667.0,+34.0225015,-118.1996129,Non-Violent
1004988,251304095,01/31/2025 12:00:00 AM,01/30/2025 12:00:00 AM,1554,13,Newton,1372,2,850,INDECENT EXPOSURE,...,90011,860Z200US90011,B5,G6350,S,11108402.0,515.0,+34.0070900,-118.2586805,Non-Violent
1004989,251704066,01/17/2025 12:00:00 AM,01/17/2025 12:00:00 AM,1600,17,Devonshire,1774,2,624,BATTERY - SIMPLE ASSAULT,...,91330,860Z200US91330,B5,G6350,S,1292974.0,0.0,+34.2445219,-118.5266770,Violent


In [53]:
# Aggregate to the zip code level and desired time period
# -- For now, doing monthly

# Convert incident date to pandas datetime
incidents_zcta["incident_date"] = pd.to_datetime(incidents_zcta['DATE OCC'], format = "%m/%d/%Y %I:%M:%S %p")

# Get month/year from date
incidents_zcta["incident_month"] = incidents_zcta['crime_date'].dt.to_period('M')

In [66]:
##############################
# SUMMARIES OF ZIP CODE/AREA 
##############################
zips_per_area = incidents_zcta.groupby("AREA")["ZCTA"].nunique().reset_index()
zips_per_area = zips_per_area.rename(columns = {"ZCTA": "n_zips"})
print("ZIP codes per area:", min(zips_per_area["n_zips"]), "to", max(zips_per_area["n_zips"]))

areas_per_zip = incidents_zcta.groupby("ZCTA")["AREA"].nunique().reset_index()
areas_per_zip = areas_per_zip.rename(columns = {"AREA": "n_areas"})
print(len(areas_per_zip), "zip codes in LAPD data")

overlapping_zips = areas_per_zip[areas_per_zip["n_areas"] > 1]
print("Overlapping ZIP codes (in multiple areas):", len(overlapping_zips))

areas_with_overlaps = incidents_zcta[incidents_zcta["ZCTA"].isin(overlapping_zips["ZCTA"])]["AREA"].unique()
print("Areas with overlapping zip codes:", len(areas_with_overlaps))

ZIP codes per area: 9 to 25
150 zip codes in LAPD data
Overlapping ZIP codes (in multiple areas): 75
Areas with overlapping zip codes: 21


In [87]:
# Aggregate incident data to zip code and month
incidents_zcta_month = incidents_zcta.groupby(["ZCTA", "incident_month"])["Crime Type"].value_counts().unstack(fill_value = 0).add_prefix("crime_").reset_index()
incidents_zcta_month

#### THINGS TO FIGURE OUT
# -- Rolling window calculations
# -- Need to fill in months if they're missing for a zip code first?
#incidents_zcta_month['violent_rolling_sum_2M'] = incidents_zcta_month['crime_Violent'].rolling(window='2M', on="incident_month").sum()

Crime Type,ZCTA,incident_month,crime_Non-Violent,crime_Violent
0,90001,2020-01,48,38
1,90001,2020-02,34,12
2,90001,2020-03,30,16
3,90001,2020-04,16,25
4,90001,2020-05,31,22
...,...,...,...,...
7831,91608,2024-07,3,0
7832,91608,2024-09,2,0
7833,91608,2024-10,1,0
7834,91801,2020-01,3,0


## ACS Data

In [20]:
# Read in ACS data
acs_population_raw = pd.read_csv(raw_acs_population_path)
acs_population_raw.head()

###### GET MORE ACS FIELDS
# -- Median income
# -- poverty related fields?
# -- unemployment
# -- income inequality index 
# Not sure if these are available at zip level

,GEO_ID,NAME,B01003_001E,B01003_001M,Unnamed: 4
0,Geography,Geographic Area Name,Estimate!!Total,Margin of Error!!Total,NaN
1,0400000US06,California,39242785,*****,NaN
2,860Z200US89010,ZCTA5 89010,355,127,NaN
3,860Z200US89019,ZCTA5 89019,2355,489,NaN
4,860Z200US89060,ZCTA5 89060,12783,1225,NaN


In [88]:
# Drop first two rows
acs_population = acs_population_raw.drop(index = range(0, 2)).reset_index(drop = True)

# Drop last column
acs_population = acs_population.drop(acs_population.columns[-1], axis = 1)

# Reset column  names
acs_pop_colnames = ["geo_id", "zcta_name", "total_population", "total_population_moe"]
acs_population.columns = acs_pop_colnames

# Get zip code on it's own (keep as string to match with zcta table above)
acs_population["ZCTA"] = acs_population["zcta_name"].str.split(' ', n = 1).str[1]

# Convert population columns to numeric
acs_population["total_population"] = pd.to_numeric(acs_population["total_population"], errors = "coerce")
acs_population["total_population_moe"] = pd.to_numeric(acs_population["total_population_moe"], errors = "coerce")

# Only keep necessary columns that we want to join
# -- Not sure if we need to use margin of error?? Read into this from ACS on how/if it should be used
acs_population = acs_population[["ZCTA", "total_population", "total_population_moe"]]

acs_population.head()

,ZCTA,total_population,total_population_moe
0,89010,355,127
1,89019,2355,489
2,89060,12783,1225
3,89061,7835,967
4,89439,2071,337


## Merge ACS Data to Incidents

In [89]:
incidents_final = pd.merge(incidents_zcta_month, acs_population, on = "ZCTA", how = "left")
incidents_final

,ZCTA,incident_month,crime_Non-Violent,crime_Violent,total_population,total_population_moe
0,90001,2020-01,48,38,56403,2919
1,90001,2020-02,34,12,56403,2919
2,90001,2020-03,30,16,56403,2919
3,90001,2020-04,16,25,56403,2919
4,90001,2020-05,31,22,56403,2919
...,...,...,...,...,...,...
7831,91608,2024-07,3,0,38,30
7832,91608,2024-09,2,0,38,30
7833,91608,2024-10,1,0,38,30
7834,91801,2020-01,3,0,53730,1136


In [90]:
incidents_final.describe()

####### NEED TO LOOK INTO THAT ZERO POPULATION ONE... 

,crime_Non-Violent,crime_Violent,total_population,total_population_moe
count,7836.000000,7836.000000,7836.000000,7836.000000
mean,86.264931,41.650459,35457.618300,1783.428535
std,69.250862,47.404689,20559.098244,794.596397
min,0.000000,0.000000,0.000000,14.000000
25%,27.000000,3.000000,21569.000000,1335.000000
50%,80.000000,26.000000,32536.000000,1796.000000
75%,129.000000,66.000000,47303.000000,2151.000000
max,522.000000,296.000000,102784.000000,4421.000000
